In [19]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

fatal: destination path 'cbdb_sqlite' already exists and is not an empty directory.


In [20]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


In [21]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

In [22]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [23]:
addr_df_c_name_chn = pd.read_sql_query("SELECT c_name_chn FROM ADDR_CODES", CONN).drop_duplicates().dropna()
addr_df_c_alt_names = pd.read_sql_query("SELECT c_alt_names FROM ADDR_CODES", CONN).drop_duplicates().dropna()
addr_df = pd.concat([addr_df_c_name_chn, addr_df_c_alt_names], axis=1).stack().reset_index(drop=True).drop_duplicates().dropna().to_frame(name='addr_chn')
addr_df

,addr_chn
0,[信息缺乏]
1,[未詳]
2,
3,中華人民共和國
4,北京省市
...,...
13374,定戎
13375,義合
13376,積石軍
13377,敷文


In [24]:
# read in address type csv file
address_type_df = pd.read_csv('../cbdb_entity_address_types.csv')
address_type_list = address_type_df['name'].to_list()
address_type_list = address_type_list + ["等", "等處"]
address_type_list = sorted(address_type_list, key=len)
address_type_list.reverse()
print(len(address_type_list))

['都團練觀察使', '備禦右千戶所', '備禦前千戶所', '備禦後千戶所', '備禦左千戶所', '都團練觀察', '江防兵備道', '軍民總管府', '觀察防遏使', '防御觀察使', '團練觀察使', '副都統轄區', '特別行政區', '行政督查區', '群牧千戶所', '備禦千戶所', '左右千戶所', '中後千戶所', '中右千戶所', '中左千戶所', '中前千戶所', '軍民千戶所', '守禦千戶所', '沿邊第十區', '沿邊第九區', '沿邊第八區', '沿邊第七區', '沿邊第六區', '沿邊第五區', '沿邊第四區', '沿邊第三區', '沿邊第二區', '沿邊第一區', '觀察防遏', '防御觀察', '團練觀察', '廉訪司道', '封國封郡', '行中書省', '都防御使', '將軍轄區', '省諸道區', '右千戶所', '左千戶所', '前千戶所', '都防御', '百戶所', '千戶所', '分守道', '分巡道', '監察道', '監軍道', '兵備道', '羈縻州', '自治州', '總帥府', '都護府', '總管府', '都督府', '自治縣', '直隸廳', '鎮守軍', '道行軍', '守捉使', '處置使', '經略使', '防御使', '團練使', '觀察使', '節度使', '行政區', '特別區', '綏靖區', '市轄區', '群牧所', '宣撫司', '布政司', '慰使司', '宣慰司', '儀衛司', '安撫司', '揮使司', '長官司', '中護衛', '左護衛', '右護衛', '前屯衛', '後屯衛', '中屯衛', '右屯衛', '等處', '守捉', '經略', '防御', '團練', '觀察', '中道', '諸道', '東道', '西道', '王府', '土縣', '分縣', '左旗', '右旗', '後旗', '前旗', '中旗', '府路', '北路', '南路', '西路', '東路', '郊區', '特區', '地區', '節度', '護衛', '前衛', '中衛', '左衛', '右衛', '五衛', '等', '縣', '省', '廳', '軍', '旗', '路', '使', '區', '司', '市', '衛', '道', '郡', '州', '府']
139


In [25]:
def recursively_remove_addr_type_from_text(res, text, address_type_list):
  if not text:
    return
  res.append(text)
  for word in address_type_list:
    if text.endswith(word):
      remains = text[:len(text)-len(word)]
      recursively_remove_addr_type_from_text(res, remains, address_type_list)
      break;

def remove_addr_type_from_text(text, address_type_list):
  res = []
  recursively_remove_addr_type_from_text(res, text, address_type_list)
  return res

In [26]:
# Test get_addr_tokens_from_text
input = "沿邊第一區備禦後千戶所左衛"
res = remove_addr_type_from_text(input, address_type_list)
print(res)

['沿邊第一區備禦後千戶所左衛', '沿邊第一區備禦後千戶所', '沿邊第一區']


In [27]:
def names_massage(df, column):
  # split line by ;
  df[column] = df[column].apply(lambda x: x.split(';'))
  df = df.explode(column).drop_duplicates().dropna()
  # remove row with 、
  df = df[df[column].str.contains('、')==False]
  # replace spaces with blank
  df[column] = df[column].str.replace("\\s", "", regex=True)
  # remove single-character row
  df = df[df[column].apply(lambda x: len(x)>1)]
  # remove the addresses which contain any English letters or any sort of brackets
  df = df[df[column].str.contains('.*[a-zA-Z]+.*')==False]
  df = df[df[column].str.contains('\(|\[|{|\)|\]|}|（|「|【|）|」|】')==False]
  # remove address type recursively
  df[column] = df[column].apply(lambda x: remove_addr_type_from_text(x, address_type_list))
  df = df.explode(column).drop_duplicates().dropna()
  # remove single-character 
  df = df[df[column].apply(lambda x: len(x)>1)]
  # remove row that's too long
  df = df[df[column].apply(lambda x: len(x)<14)]
  return df

In [28]:
filtered_addr_df = names_massage(addr_df, 'addr_chn')

In [29]:
filtered_addr_df

,addr_chn
3,中華人民共和國
4,北京省市
4,北京省
4,北京
5,天津省市
...,...
13375,義合
13376,積石軍
13377,敷文
13378,江淮等路


In [30]:
# Sort by descending order of the length
final_df = filtered_addr_df.sort_values(by="addr_chn", key=lambda x: -x.str.len())
final_df

,addr_chn
9777,朵甘思田地裏管軍民都元帥府
7609,磨坡雷波等處蠻夷軍民長官司
7501,沿河祐溪等處蠻夷軍民長官司
10316,固原靜寧隆德鎮原等處兵備道
7607,葛蠻雍真等處蠻夷軍民長官司
...,...
3554,定武
3555,賓州
3556,遷民
3527,嘉州


In [31]:
final_df.to_csv('cbdb_entity_addresses.csv', index=False)

In [32]:
# close the db
CONN.close()